# Proyecto Final

***Equipo 07***

- Aide Jazmín González Cruz
- Elena Villalobos Nolasco
- Carolina Acosta Tovany

#### Instrucciones

El proyecto/examen final consistirá en:

La implementación del algoritmo de filtrado colaborativo utilizando la metodología vista en clase (uso de otra metodología no se calificará).

Todos los algoritmos de aprendizaje de máquina que se utilicen deberán haber sido creados por ustedes. Sólo podrán utilizar Transformers y funciones de apoyo de scikit-learn (para realizar la división de los datos en entrenamiento y prueba, o el procedimiento de validación cruzada, etc.) mas ningún estimator (regresión logística, máquina de vectores de soporte, k medias, etc.). 

Se deberá explicar como se obtuvo la k con la que se generó el resultado final.

Se utilizarán los archivos con el conjunto pequeño de calificaciones y películas ubicado en la siguiente https://www.kaggle.com/rounakbanik/the-movies-dataset:

- **links_small.csv**: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

- **ratings_small.csv**: The subset of 100,000 ratings from 700 users on 9,000 movies.

Con el fin de mejorar la calificación (opcional, puntos extra), se podrán utilizar los algoritmos desarrollado en las tareas del curso y los datos relevantes (los que hacen match con los datos anteriores) contenidos en los archivos:

- **movies_metadata.csv**: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

- **keywords.csv**: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

- **credits.csv**: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

La métrica con la que se determinará el desempeño del algoritmo es el NDCG 

(https://en.wikipedia.org/wiki/Discounted_cumulative_gain#Normalized_DCG)

Una vez obtenida la matriz de calificaciones, el programa deberá ser capaz de regresar las 5 mejores recomendaciones del o de los usuarios que se consulten.

El proyecto se entregará en un Jupyter notebook. El readme file debe contener las instrucciones para que se ejecute el código. Deben cerciorarse que siguiendo esas instrucciones el programa corre sin errores. 

Se deberá subir a la carpeta proyecto_final/equipo_xx en el repositorio GitHub antes de las 7:00 am del día del examen final (14 de diciembre de 2020).    

In [1]:
# Importación de paqueterías necesarias
import pandas as pd
import numpy as np
import random
from sympy import solve
import sympy
from sympy.tensor.array import derive_by_array
from scipy.optimize import fsolve

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [3]:
# Importación de datos
links_small = pd.read_csv('links_small.csv')
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
links_small.shape

(9125, 3)

In [5]:
ratings_small = pd.read_csv('ratings_small.csv')
ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
ratings_small.shape

(100004, 4)

In [7]:
# Películas en catálogo que no han calificado los usuarios
df_mov_u = pd.DataFrame(ratings_small['movieId'])
df_mov = pd.DataFrame(links_small['movieId'])

In [8]:
common = df_mov.merge(df_mov_u, on=["movieId"])
common

,movieId
0,1
1,1
2,1
3,1
4,1
...,...
99999,161944
100000,162376
100001,162542
100002,162672


In [9]:
result = df_mov[~df_mov.movieId.isin(common.movieId)]
result.shape

(59, 1)

In [10]:
# Construyendo la matriz Y_ai
y_ia = links_small.set_index('movieId').join(ratings_small.set_index('movieId'))
y_ia = y_ia.reset_index()
y_ia

,movieId,imdbId,tmdbId,userId,rating,timestamp
0,1,114709,862.0,7.0,3.0,8.518667e+08
1,1,114709,862.0,9.0,4.0,9.386292e+08
2,1,114709,862.0,13.0,5.0,1.331380e+09
3,1,114709,862.0,15.0,2.0,9.979383e+08
4,1,114709,862.0,19.0,3.0,8.551901e+08
...,...,...,...,...,...,...
100058,162672,3859980,402672.0,611.0,3.0,1.471524e+09
100059,163056,4262980,315011.0,NaN,NaN,NaN
100060,163949,2531318,391698.0,547.0,5.0,1.476419e+09
100061,164977,27660,137608.0,NaN,NaN,NaN


In [11]:
y_ia['movieId'].nunique()

9125

In [12]:
max(y_ia.rating)

5.0

In [13]:
#y_ia.pivot(index="userId", columns="movieId", values="rating") 
y_ia = pd.DataFrame(y_ia.pivot(index='userId', columns='movieId', values='rating'))
y_ia = pd.DataFrame(y_ia.to_records())
y_ia = y_ia[pd.notnull(y_ia['userId'])]
y_ia['userId'] = y_ia['userId'].astype(int)
y_ia = y_ia.drop(['userId'], axis=1)
y_ia

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
users, movies = y_ia.shape

La función objetivo:
    
$$J(X) = \frac{1}{2} \displaystyle\sum_{(a,i)\in\mathbb{D}} \left(Y_{ai}-\left [ UV^T \right ]_{ai} \right)^2 + \frac{\lambda}{2} \displaystyle\sum_{a=1}^n \displaystyle\sum_{j=1}^k U_{aj}^2 + \frac{\lambda}{2} \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k V_{ij}^2$$

Tenemos una $k = 1$

In [15]:
# Fijando semilla
random.seed(0)
# Creando el vector V que son las películas al azar
V = np.random.randint(1,9,size = (1,movies))
V.shape

(1, 9125)

In [16]:
# Sacando el vector U que son los usuarios
U = np.random.randint(1,9,size = (1,users))
U.shape

(1, 671)

In [17]:
a =  np.array([[ 2,  7,  8]])
b = a*a.T
b = 1/b
sum(b)


array([0.384, 0.11 , 0.096])

In [18]:
yai = np.array([[5,0, 7.0],
                [1,   2,0]])
yai

array([[5., 0., 7.],
       [1., 2., 0.]])

In [19]:
v = np.array([2, 7, 8])
v

array([2, 7, 8])

In [20]:
u = np.array([2,2])
u

array([2, 2])

In [21]:
# Producto exterior
np.outer(u,v)

array([[ 4, 14, 16],
       [ 4, 14, 16]])

In [22]:
u1, u2 = sympy.symbols("u1, u2")

In [23]:
fo_u1 = (5-2*u1)**2/2 + (7-8*u1)**2/2 + 1/2 * u1**2

In [24]:
fo_u1

0.5*u1**2 + (5 - 2*u1)**2/2 + (7 - 8*u1)**2/2

In [25]:
gf_u1 = derive_by_array(fo_u1, (u1))

In [26]:
gf_u1

69.0*u1 - 66

In [27]:
solve(gf_u1)

[0.956521739130435]

In [28]:
fo_u2 = (1-2*u2)**2/2 + (2-7*u2)**2/2 + 1/2*u2**2
fo_u2

0.5*u2**2 + (1 - 2*u2)**2/2 + (2 - 7*u2)**2/2

In [29]:
gf_u2 = derive_by_array(fo_u2, (u2))
gf_u2

54.0*u2 - 16

In [30]:
solve(gf_u2)

[0.296296296296296]

In [31]:
ula = np.array([solve(gf_u1),solve(gf_u2)])

In [32]:
ula

array([[0.956521739130435],
       [0.296296296296296]], dtype=object)

In [47]:
movies_metadata = pd.read_csv('movies_metadata.csv',low_memory=False)
movies_metadata.shape

(45466, 24)

In [49]:
movies_metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [50]:
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [148]:
movies_metadata['imdb_id'].nunique()

45417

In [52]:
movies_metadata['title'].nunique()

42277

In [149]:
movies_metadata['id'].nunique()

45436

In [63]:
df_mov_u = pd.DataFrame(ratings_small['movieId'])
df_mov_u.nunique()

movieId    9066
dtype: int64

In [265]:
df_mov = pd.DataFrame(links_small[['tmdbId','movieId']])
df_mov.head(50)

,tmdbId,movieId
0,862.0,1
1,8844.0,2
2,15602.0,3
3,31357.0,4
4,11862.0,5
5,949.0,6
6,11860.0,7
7,45325.0,8
8,9091.0,9
9,710.0,10


In [267]:
hola1 = pd.DataFrame(movies_metadata[['id','title']])
hola1 = hola1.rename(columns = {'id':'tmdbId'})
hola1.head(50)

,tmdbId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II
5,949,Heat
6,11860,Sabrina
7,45325,Tom and Huck
8,9091,Sudden Death
9,710,GoldenEye


In [268]:
hola1.nunique()

tmdbId    45436
title     42277
dtype: int64

In [270]:
malos = hola1[hola1['tmdbId'].str.contains('-')].index

In [271]:
hola1 = hola1.drop(index=malos)

In [273]:
hola1['tmdbId']=hola1['tmdbId'].apply(lambda x: float(x))

In [278]:
df_mov.isna().sum()

tmdbId     13
movieId     0
dtype: int64

In [279]:
df_mov=df_mov.dropna(0)

In [280]:
id_base=df_mov.sort_values(by=['tmdbId'])

In [281]:
titles_id= hola1.sort_values(by=['tmdbId'])

In [286]:
bases_nombres_id=id_base.merge(titles_id, on=['tmdbId'],how='left')

In [289]:
movies = np.array([4470,18,479,260,6377])

In [300]:
bases_nombres_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9129 entries, 0 to 9128
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   tmdbId   9129 non-null   float64
 1   movieId  9129 non-null   int64  
 2   title    9099 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 285.3+ KB


In [302]:
bases_nombres_id

,tmdbId,movieId,title
0,2.0,4470,Ariel
1,5.0,18,Four Rooms
2,6.0,479,Judgment Night
3,11.0,260,Star Wars
4,12.0,6377,Finding Nemo
...,...,...,...
9124,402672.0,162672,Mohenjo Daro
9125,410612.0,162376,NaN
9126,410803.0,164979,NaN
9127,410921.0,875,Nothing to Lose


In [294]:
movies

array([4470,   18,  479,  260, 6377])

In [314]:
bases_nombres_id[bases_nombres_id['movieId']==875]

,tmdbId,movieId,title
9127,410921.0,875,Nothing to Lose


In [320]:
bases_nombres_id[bases_nombres_id['movieId'].isin(movies)]

,tmdbId,movieId,title
0,2.0,4470,Ariel
1,5.0,18,Four Rooms
2,6.0,479,Judgment Night
3,11.0,260,Star Wars
4,12.0,6377,Finding Nemo
